<h1>THE WHAT INFLUENCES ARE IN THE AUTO INDUSTRY</h1>


<img src =https://n6-img-fp.akamaized.net/free-vector/petrol-station-design_1454-41.jpg?size=338&ext=jpg>

<h3>MOTIVATION</h3>

<p>When thinking about what type of final project we wanted to do, we wanted a topic that we can extract years of data from as well as be a topic that applies to current times. What we have selected was the Fuel economy data that contained fuel economy data from as far as 1978 but for this project we'll be focusing on years 2001 - 2017. The reason we used this time range is due to only wanting to use data within the 2000s while having enough data for formulate more accurate findings. With this data, we also wanted to have a correlations with the different categories like manufacturer, classification of the cars in the dataset as well as fuel cost and such. By the end of this tutorial, you'll find out what type of influence fuel pays a part in the auto industry while using past data to make a educated guess about the future for the current year.
</p>

<h3> THE CHALLENGES AHEAD </h3>

<p>We have chosen a website such that data is vast and filled with subjects that we can use. This was a double edged sword however because even though we had a lot of data to work with, we had to decide with data to not use and which data to we can use to benefit us. One of the biggest problems we ran into was that for our data set, the classification would change in spelling despite representing the same thing as pervious years or for the more recent datasets that we scrapped new classifications were created. With such a mix /mash of data varying in time periods, we had to find the similairities and use the common data in favor of us for this project.</p>

<h3>*\BE SURE TO HAVE LXML INSTALLED ON YOUR NOTEBOOK OR ELSE THE READIND TIDYING OF THE DATA WON'T WORK*/</h3>

In [56]:
!pip install lxml
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np
import zipfile
import os
import shutil

import matplotlib.pyplot as plt
import seaborn as sns



<p>Open url for scraping. All data sources are stored within a table. All target data files have the
same naming scheme, we will take advantage of this</p>

In [57]:

url = 'http://www.fueleconomy.gov/feg/download.shtml'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')


reg = 'epadata/\d{2}data.zip' ##luckily our target has a unique identifier for href
suf = 'http://www.fueleconomy.gov/feg/'

<p>Find target data sources within html page</p>

In [58]:
links = soup.find_all('a')
zlist = []
for tag in links:
    link = tag.get('href',None)
    if link is not None:
        if re.match(reg, link):
            zlist.append(link)

<p>Download data to current working directory. These files are zipped. Unzip files</p>

In [59]:
filelst = []
for x in zlist:
    url = suf+x
    localfn = url.split('/')[-1]
    yearno = int(localfn[:2])
    if yearno < 50 and yearno > 00:
        r = requests.get(url, stream=True)
        with open(localfn, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
        zip_ref = zipfile.ZipFile(localfn, 'r')
        lcl = "dldir"+localfn
        filelst.append(lcl)
        zip_ref.extractall(lcl)
        zip_ref.close()
       
        


<p>create directory to store data</p>

In [60]:
datdir = "shibe_data"
if not os.path.exists(datdir):
    os.makedirs(datdir)

<p>extract downloaded data to single directory. Rename files so that they have uniform names</p>

In [61]:
reg = "(.xlsx|.xls|.csv)$"
for item in filelst:
    for filename in os.listdir(item):
        curname =  item + "\\" + filename 
        
        
        m = re.search(reg, filename)
        if m:
            newname = datdir + "\\" + item[:-4] + m.group(0)
            os.rename(curname, newname)


FileNotFoundError: [Errno 2] No such file or directory: 'dldir18data.zip\\2018 FE Guide for DOE-release dates before 12-14-2017-no-sales-12-12-2017public.xlsx' -> 'shibe_data\\dldir18data.xlsx'

<p>clean up files and directories created while scraping data</p>

In [ ]:

lst = os.listdir(os.getcwd())
for x in lst:
    m = re.search(".zip$", x)
    if m:
        n = re.search("^dldir",x)
        if n:
            shutil.rmtree(x)
        else:
            os.remove(x)


<p> Reads data files into Pandas Dataframes according to file type (.csv .xlx .xlsx). The df are stored in a temporary dict according to their respective years. The original zip file from the EPA site does not give year in YYYY format, so we have chosen not to use the YYYY format until this point where it becomes necessary and after we have gleaned out only the years prefixed by "20-"Some lines of the original data files are formed incorrectly. Adding the option "error_bad_lines=False" skips these lines<p>

In [ ]:
%%capture
datfiles = os.listdir(datdir)
pdfiles = {}
for x in datfiles:
    y = re.search("[0-9]{2}",x)
    year = "20" + y.group(0)
    cs = re.search(".csv$", x)
    if cs:
        newp = pd.read_csv(datdir + "\\" + x, error_bad_lines=False)
    else:
        newp = pd.read_excel(datdir + "\\" + x)
        
    pdfiles[year] = newp
    

<p>Went through all the data files from 2001-2018 and found all the persisting variables and listed their aliases in a dict, vars</p>

In [ ]:
varss = {}
varss['Class'] = ['CLASS','Carline Class Desc', 'Class']
varss['Manufacturer'] = ['MFR', 'Manufacturer', 'Mfr Name', 'Mfr Name ']
varss['carline name'] = ['CAR LINE', 'Carline', 'carline name']
varss['cmb'] = ['COMB MPG (GUIDE)', 'Comb FE (Guide) - Conventional Fuel', 'cmb']
varss['cty'] = ['CITY MPG (GUIDE)', 'City FE (Guide) - Conventional Fuel', 'cty']
varss['cyl'] = ['# Cyl', 'NUMB CYL', 'cyl']
varss['displ'] = ['DISPLACEMENT', 'Eng Displ', 'displ']
varss['fcost'] = ['ANL FL CST', 'Annual Fuel1 Cost - Conventional Fuel', 'fcost']
varss['fl'] = ['FUEL TYPE', 'Fuel Usage Desc - Conventional Fuel', 'fl']
varss['hwy'] = ['HWY MPG (GUIDE)', 'Hwy FE (Guide) - Conventional Fuel', 'hwy']
varss['trans'] = ['TRANS', 'Trans as listed in FE Guide (derived from col AA thru AF)', 'Trans in FE Guide (MFR entered for data entered after May 13 2011)', 'Transmission', 'trans']
varss['ucmb'] = ['Comb Unadj FE - Conventional Fuel', 'UNRND COMP (EPA)', 'ucmb']
varss['ucty'] = ['City Unadj FE - Conventional Fuel', 'UNRND CITY (EPA)', 'ucty']
varss['uhwy'] = ['Hwy Unadj FE - Conventional Fuel', 'UNRND HWY (EPA)', 'uhwy']
varss['drv'] = ['DRIVE SYS', 'Drive Sys', 'drv']

<p> Finds all the column names which are included inside of the shared variable map and replaces the dataframe 
with only those columns. Renames them to the shared variable map's key names, so that column names can be 
standardized. The columns are reordered before replacing the dataframes inside the original master map of df</p>

In [ ]:
varlst = list(varss.keys())
yearlst = list(pdfiles.keys())


for z in yearlst:
    curpd = pdfiles[z]
    cols = list(curpd.columns.values)
    checklst = []
    rename = []
    
    for x in cols:
        for y in varlst:
            if x in varss[y]:
                checklst.append(x)
                rename.append(y)
    curpd = curpd[checklst]
    curpd.columns = rename
    curpd = curpd[varlst]
    curpd['year'] = z
    pdfiles[z] = curpd


<p>Finds all car classes from over the years. Aliases for car classes are 
combined in the carclass dict. Some types (small SUV and standard SUV are 
combined since they were originally the same category in previous years).<p>

In [ ]:
cartypes = []
for x in yearlst:
    curpd = pdfiles[x]
    temp = set(curpd['Class'].values)
    cartypes.extend(list(temp))
cartypes = list(set(cartypes))

In [ ]:
carclass = {}

carclass['COM'] = ['COMPACT CARS','Compact Cars','SUBCOMPACT CARS','Subcompact Cars','MINICOMPACT CARS','Minicompact Cars']
carclass['LAR'] = ['LARGE CARS','Large Cars']
carclass['MID'] = [ 'MIDSIZE CARS','Midsize Cars']
carclass['MINIV'] = ['MINIVAN - 2WD','Special Purpose Vehicle, minivan 2WD','SPEC PURP VEH - MINIVAN - 2WD','SPEC PURP VEH - MINIVAN - 4WD','Special Purpose Vehicle, minivan 4WD','MINIVAN - 4WD']
carclass['PU'] = ['SMALL PICKUP TRUCKS 4WD','Small Pick-up Trucks 4WD', 'Small Pick-up Trucks 2WD','STANDARD PICKUP TRUCKS 2WD','Standard Pick-up Trucks 2WD','Standard Pick-up Trucks 4WD']

carclass['SW'] = ['MIDSIZE STATION WAGONS','SMALL STATION WAGONS','Small Station Wagons','Midsize Station Wagons']
carclass['SUV'] = ['SPEC PURP VEH - S.U.V. - 4WD','S.U.V. - 4WD','Small SUV 4WD','Special Purpose Vehicle, SUV 4WD','Standard SUV 4WD','SPEC PURP VEH - S.U.V. - 2WD','S.U.V. - 2WD','Small SUV 2WD','Special Purpose Vehicle, SUV 2WD','Standard SUV 2WD']

carclass['2S'] = ['TWO SEATERS','Two Seaters']


In [ ]:
pig = sns.light_palette("green",18)
piv = sns.light_palette("purple",18)
pir= sns.light_palette("red",18)


<p> You can are going to get retrieve any 10 manufacturers and store them in an array that can be later used for accessing a subset of the data given to us. By doing this you can start being to isolate the data per manufacturer and to see if there's any evident correlation or more data to be found. The same applies to the class of the car but we'll use ,4 unlike the manufacturer. There are more than 10 manufacturers and 4 class of cars in the data set but we want to zoom in to certain classifications and see if the correlation is manufacturer dependent or if the data is the same or similar across the board.</p>

In [ ]:
manufacturer = ['toyota', 'bmw', 'fca us llc','general motors', 'kia', 'nissan', 'honda', 'mercedes-benz', 'volkswagen']
classs = ['COM','SUV','PU','MINIV']

<p>The code below is a loop that prints a scatter plot to represent each manufacturer (that was randomly selected due to their popularity).and the correlation that each manufacturer has between the annual fuel cost the the City (MPG)</p>


In [ ]:
for brands in manufacturer:
    fig = plt.figure()
    plt.title(brands)
    plt.xlabel('Annual Fuel Cost')
    plt.ylabel('CITY (MPG)')
    for x in yearlst:
        curpd = pdfiles[x]
        th = curpd[curpd['Manufacturer'].str.lower() == brands]
        i = th['fcost']
        j = th['cty']
        plt.scatter(i,j, color = pin[int(x)-2001],label = x)
        
    plt.legend()
    plt.show()

<p>The code below does the same as the code above, the only difference is that now we're seeing a correlation between annual fuel cost and the Highway (MPG)'</p>

In [ ]:
for brands in manufacturer:
    fig = plt.figure()
    plt.title(brands)
    plt.xlabel('Annual Fuel Cost')
    plt.ylabel('HIGHWAY (MPG)')
    for x in yearlst:
        curpd = pdfiles[x]
        th = curpd[curpd['Manufacturer'].str.lower() == brands]
        i = th['fcost']
        j = th['hwy']
        plt.scatter(i,j, color = pin[int(x)-2001],label = x)
        
    plt.legend()
    plt.show()
    

 <p>The code below does the same as the code above, the only difference is that now we're seeing a correlation between annual fuel cost and the combined MPG of the Highway and the City'</p>

In [ ]:
for brands in manufacturer:
    fig = plt.figure()
    plt.title(brands)
    plt.xlabel('Annual Fuel Cost')
    plt.ylabel('HIGHWAY (MPG)')
    for x in yearlst:
        curpd = pdfiles[x]
        th = curpd[curpd['Manufacturer'].str.lower() == brands]
        i = th['fcost']
        j = th['cmb']
        plt.scatter(i,j, color = pin[int(x)-2001],label = x)
        
    plt.legend()
    plt.show()

<p>You are now going to use the same logic as before but now you'll be using the deriving data from the class of cars. We are testing things out and trying to find a relation between any of the data given to us. As you walk through this, even you yourself will see a pattern in which you'll start putting the data together to make a deduction of what the data is telling us.</p>

In [ ]:
fig = plt.figure()
plt.xlabel('Annual Fuel Cost')
plt.ylabel('highway (MPG)')
plt.title('Compact car analysis')
for x in yearlst:
    curpd = pdfiles[x]
    s = set(curpd['Class'].values)
    for y in s:
        if y in carclass['COM']:
            th = curpd[curpd['Class'] == y]
            i = th['fcost']
            j = th['hwy']
            plt.scatter(i,j, color = pig[int(x)-2001])

plt.legend()
plt.show()    

In [ ]:
fig = plt.figure()
plt.xlabel('Annual Fuel Cost')
plt.ylabel('city (MPG)')
plt.title('Compact car analysis')
for x in yearlst:
    curpd = pdfiles[x]
    s = set(curpd['Class'].values)
    for y in s:
        if y in carclass['COM']:
            th = curpd[curpd['Class'] == y]
            i = th['fcost']
            j = th['cty']
            plt.scatter(i,j, color = piv[int(x)-2001],label = x)

plt.legend()
plt.show()

In [ ]:
fig = plt.figure()
plt.xlabel('Annual Fuel Cost')
plt.ylabel('combined (MPG)')
plt.title('Compact car analysis')
for x in yearlst:
    curpd = pdfiles[x]
    s = set(curpd['Class'].values)
    for y in s:
        if y in carclass['COM']:
            th = curpd[curpd['Class'] == y]
            i = th['fcost']
            j = th['cmb']
            plt.scatter(i,j, color = pir[int(x)-2001],label = x)

plt.legend()
plt.show()

<p>From the above plots, annual fuel cost has a clear relationship with 
mileage per gallon. Lower fuel cost corresponds to higher mpg, while the 
reverse happens for high annual fuel cost. Another observation is that 
annual fuel cost increases over the late 00s and then decreases over the years.</p>

In [ ]:
fig = plt.figure()
plt.xlabel('year')
plt.ylabel('Annual fuel cost')
plt.title('Annual Fuel Cost over the years')
byear = []
bcost = []
for x in yearlst:
    curpd = pdfiles[x]
    byear.extend(curpd['year'])
    bcost.extend(curpd['fcost'])
    
    plt.scatter(curpd['year'],curpd['fcost'])
b = pd.DataFrame(byear,columns=['year'])
b["cost"] = bcost

avg_lifeExp_per_interval = b.groupby('year').mean()
avg_lifeExp_per_interval.cost.plot()
plt.show()

sns.violinplot(y=bcost, x=byear)
plt.show()

<p>The above graph and violin plot shows the change in annual fuel cost over the years across all car 
 classes and car manufacturers. This may point to the fact that the annual fuel cost may have 
 underlying factors besides the car maker and car types. One potential source could be the inflation of 
 gas prices centered around 2009.</p>  

Tested and Approved by: Chris Chan and Edward N Mancho